In [1]:
# @title 1. 환경 설정 및 모델 로드 (실행 후 잠시 대기)
!pip install ultralytics -q
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# YOLOv8-pose 모델 다운로드 및 로드 (가장 가벼운 nano 버전 사용)
model = YOLO('yolov8n-pose.pt')
print("✅ 모델 로드 완료! 다음 셀을 실행해주세요.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ 모델 로드 완료! 다음 셀을 실행해주세요.


In [2]:
# @title 2. 분석용 수학 함수 정의

def calculate_angle(a, b, c):
    """
    세 점 a, b, c 사이의 각도를 계산합니다 (b가 꼭짓점).
    수학적 원리: 벡터의 내적 공식 또는 아크탄젠트(atan2) 활용
    """
    a = np.array(a) # 첫 번째 점 (예: 어깨)
    b = np.array(b) # 꼭짓점 (예: 팔꿈치)
    c = np.array(c) # 세 번째 점 (예: 손목)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def get_distance(a, b):
    """두 점 사이의 유클리드 거리 계산"""
    return np.linalg.norm(np.array(a) - np.array(b))

print("✅ 분석 함수 정의 완료!")

✅ 분석 함수 정의 완료!


In [3]:
# @title 3. AI 스윙 분석기 엔진 (핵심 로직)

class GolfSwingAnalyzer:
    def __init__(self):
        self.feedback = {
            "어드레스": [],
            "백스윙": [],
            "임팩트": [],
            "팔로스루": []
        }
        # 상태 추적을 위한 변수들
        self.landmarks = None
        self.initial_nose_x = None
        self.max_wrist_y = 0 # 백스윙 탑 감지용 (화면 좌표계상 y값이 작을수록 높음)
        self.min_wrist_y = 10000 # 임팩트 감지용
        self.swing_phase = "준비"
        self.analyzed_phases = set()

    def set_landmarks(self, landmarks):
        # YOLO 출력 형식에 맞춰 키포인트 좌표 변환
        self.kpts = {
            "nose": landmarks[0][:2],
            "left_shoulder": landmarks[5][:2], "right_shoulder": landmarks[6][:2],
            "left_elbow": landmarks[7][:2],    "right_elbow": landmarks[8][:2],
            "left_wrist": landmarks[9][:2],    "right_wrist": landmarks[10][:2],
            "left_hip": landmarks[11][:2],     "right_hip": landmarks[12][:2],
            "left_ankle": landmarks[15][:2],   "right_ankle": landmarks[16][:2]
        }

    def analyze_frame(self, frame_idx):
        # 간단한 상태 머신으로 스윙 단계 감지
        wrist_avg_y = (self.kpts["left_wrist"][1] + self.kpts["right_wrist"][1]) / 2
        shoulder_avg_y = (self.kpts["left_shoulder"][1] + self.kpts["right_shoulder"][1]) / 2

        # 1. 어드레스 감지 (초기 30프레임 부근)
        if frame_idx == 30 and "어드레스" not in self.analyzed_phases:
            self.swing_phase = "어드레스"
            self.initial_nose_x = self.kpts["nose"][0]
            self.analyze_address()
            self.analyzed_phases.add("어드레스")

        # 2. 백스윙 탑 감지 (손목이 어깨보다 높고, 가장 높은 지점일 때)
        elif self.swing_phase == "어드레스" and wrist_avg_y < shoulder_avg_y:
             if wrist_avg_y < self.min_wrist_y:
                 self.min_wrist_y = wrist_avg_y # 가장 높은 위치 갱신 중
             else:
                 # 손이 다시 내려오기 시작하면 탑으로 간주
                 self.swing_phase = "백스윙"
                 if "백스윙" not in self.analyzed_phases:
                    self.analyze_backswing()
                    self.analyzed_phases.add("백스윙")

        # 3. 임팩트 감지 (백스윙 후 손이 가장 낮은 지점)
        elif self.swing_phase == "백스윙" and wrist_avg_y > shoulder_avg_y:
            if wrist_avg_y > self.max_wrist_y:
                self.max_wrist_y = wrist_avg_y # 가장 낮은 위치 갱신 중
            else:
                 # 다시 올라가기 시작하면 임팩트 지점으로 간주
                 self.swing_phase = "임팩트"
                 if "임팩트" not in self.analyzed_phases:
                    self.analyze_impact()
                    self.analyzed_phases.add("임팩트")

        # 4. 팔로스루 (임팩트 이후 다시 손이 높아짐)
        elif self.swing_phase == "임팩트" and wrist_avg_y < shoulder_avg_y:
             self.swing_phase = "팔로스루"
             if "팔로스루" not in self.analyzed_phases:
                self.analyze_followthrough()
                self.analyzed_phases.add("팔로스루")


    # --- 단계별 진단 로직 ---
    def analyze_address(self):
        # 진단 1: 스탠스 너비 확인 (어깨 너비 기준)
        shoulder_width = get_distance(self.kpts["left_shoulder"], self.kpts["right_shoulder"])
        ankle_width = get_distance(self.kpts["left_ankle"], self.kpts["right_ankle"])
        if ankle_width > shoulder_width * 1.5:
            self.feedback["어드레스"].append("⚠️ [문제점] 스탠스가 너무 넓습니다. 회전이 제한될 수 있습니다.")
            self.feedback["어드레스"].append("💡 [해결책] 발을 어깨 너비 정도로 좁혀보세요.")

    def analyze_backswing(self):
        # 진단 1: 머리 고정 (스웨이) 확인
        current_nose_x = self.kpts["nose"][0]
        # 어깨 너비의 20% 이상 움직이면 스웨이로 간주
        shoulder_width = get_distance(self.kpts["left_shoulder"], self.kpts["right_shoulder"])
        if abs(current_nose_x - self.initial_nose_x) > shoulder_width * 0.2:
             self.feedback["백스윙"].append("⚠️ [문제점] 머리가 타겟 반대 방향으로 너무 많이 이동했습니다 (스웨이).")
             self.feedback["백스윙"].append("💡 [해결책] 머리를 어드레스 위치에 고정한 채 제자리에서 회전하는 느낌을 가져보세요.")

        # 진단 2: 왼팔 치킨윙 (오른손잡이 기준)
        left_arm_angle = calculate_angle(self.kpts["left_shoulder"], self.kpts["left_elbow"], self.kpts["left_wrist"])
        if left_arm_angle < 150: # 180도가 쭉 편 상태
             self.feedback["백스윙"].append(f"⚠️ [문제점] 탑에서 왼팔이 많이 구부러졌습니다 (현재 각도: 약 {int(left_arm_angle)}도).")
             self.feedback["백스윙"].append("💡 [해결책] 왼팔을 타겟 반대 방향으로 멀리 밀어내는 느낌으로 쭉 펴주세요.")

    def analyze_impact(self):
        # 진단 1: 머리 위치 확인 (뒤에 남아있는지)
        current_nose_x = self.kpts["nose"][0]
        # 타겟 방향(화면 왼쪽)으로 머리가 나갔는지 체크 (오른손잡이 정면 기준)
        if current_nose_x < self.initial_nose_x - 20: # 20 픽셀 여유
            self.feedback["임팩트"].append("⚠️ [문제점] 머리가 타겟 방향으로 덤벼들었습니다.")
            self.feedback["임팩트"].append("💡 [해결책] '공 뒤에서 친다'는 느낌으로 머리를 오른발 쪽에 남겨두세요.")

    def analyze_followthrough(self):
        # 진단 1: 왼팔 치킨윙 확인 (가장 흔한 오류)
        left_arm_angle = calculate_angle(self.kpts["left_shoulder"], self.kpts["left_elbow"], self.kpts["left_wrist"])
        if left_arm_angle < 140:
             self.feedback["팔로스루"].append("⚠️ [문제점] 임팩트 후 왼팔이 당겨지며 구부러지는 '치킨윙' 현상이 있습니다.")
             self.feedback["팔로스루"].append("💡 [해결책] 왼손등이 타겟을 보며 낮고 길게 뻗어나가는 연습을 해보세요.")

print("✅ AI 분석 엔진 준비 완료!")

✅ AI 분석 엔진 준비 완료!


In [4]:
# @title 4. 분석 시작! 영상 파일 업로드 및 실행

# 1. 파일 업로드
print("👇 아래 '파일 선택' 버튼을 눌러 정면 스윙 영상을 업로드하세요.")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

print(f"\n🎥 영상 '{video_path}' 분석을 시작합니다... 잠시만 기다려주세요.")

# 2. 비디오 처리 준비
cap = cv2.VideoCapture(video_path)
analyzer = GolfSwingAnalyzer()
frame_count = 0

# 3. 프레임별 분석 루프
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_count += 1

    # YOLOv8 추론 (사람 감지 및 포즈 추정)
    results = model(frame, verbose=False)

    # 사람이 감지되었고 키포인트가 있는 경우에만 분석
    if results[0].keypoints is not None and len(results[0].keypoints) > 0:
        # 첫 번째 감지된 사람의 키포인트 가져오기 (x, y, confidence)
        landmarks = results[0].keypoints.data[0].cpu().numpy()

        analyzer.set_landmarks(landmarks)
        analyzer.analyze_frame(frame_count)

cap.release()

# --- 최종 결과 보고서 출력 ---
display(HTML("<h2>⛳️ AI 골프 코치 분석 결과 보고서</h2>"))

phases = ["어드레스", "백스윙", "임팩트", "팔로스루"]
has_issue = False

for phase in phases:
    display(HTML(f"<h3>🔹 {phase} 단계 분석</h3>"))
    feedback_list = analyzer.feedback[phase]
    if not feedback_list:
        display(HTML("<p style='color:green;'>✅ 특별한 문제점이 발견되지 않았습니다. 훌륭합니다!</p>"))
    else:
        has_issue = True
        for item in feedback_list:
            color = "red" if "[문제점]" in item else "blue"
            display(HTML(f"<p style='color:{color};'>{item}</p>"))
    display(HTML("<hr>"))

if not has_issue:
     display(HTML("<h3>🎉 종합 평가: 완벽한 스윙입니다! 프로 데뷔를 고려해보세요.</h3>"))
else:
     display(HTML("<h3>💡 종합 평가: 위 해결책들을 의식하며 연습해보세요. 훨씬 좋은 스윙이 될 것입니다.</h3>"))

👇 아래 '파일 선택' 버튼을 눌러 정면 스윙 영상을 업로드하세요.


Saving swing.mp4.mp4 to swing.mp4.mp4

🎥 영상 'swing.mp4.mp4' 분석을 시작합니다... 잠시만 기다려주세요.
